In [6]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
from datetime import datetime

#collecting primary articles
stories = []
def getTheGoodStuff(newsstories):
    global stories
    global i
    for data in newsstories:
        htmlatag = data.find("h2", class_="title").find("a")
        headline = htmlatag.getText()
        url = htmlatag.get("href")
        d = {"headline" : headline,
             "url" : url,
            "sort_rank" : i}
        stories.append(d)
        for related in data.find_all(class_="related-item"):
            related_htmlatag = related.find("a")
            related_headline = related_htmlatag.getText()
            related_url = related_htmlatag.get("href")
            r = {"headline" : related_headline,
                 "url" : related_url,
                 "sort_rank" : i+1}
            stories.append(r)
            
def scrapeWebsites():
    global stories
    global i
    
    # Getting stories from Fox News.
    foxnews = "http://www.foxnews.com/"
    page = urllib.urlopen(foxnews)
    r  = requests.get(foxnews)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    for i in range(0, 20):
        foundstories = soup.find_all("article", class_="article story-" + str(i))
        getTheGoodStuff(foundstories)
    
scrapeWebsites()
articles = pd.DataFrame(stories)
articles.drop_duplicates(inplace=True)
articles['date'] = datetime.today().strftime('%Y-%m-%d')
articles['year'] = datetime.today().year
articles['month'] = datetime.today().month
articles['day'] = datetime.today().day
articles['hour'] = datetime.today().hour
articles['minute'] = datetime.today().minute

fox_domains = []
for web in articles.url:
    domain_items = web.split('//')[-1].split('/')[0].strip('www.').split('.')
    url_item = web.split('https://www.foxnews.com/')[-1].split('/')[0]
    if domain_items[0] in ['video','foxbusiness','nation']:
        fox_domains.append(domain_items[0])
    else:
        fox_domains.append(url_item)
articles['article_type'] = fox_domains

#collecting Opinion articles
sidebar_stories = []

def opinion_getTheGoodStuff(newsstories):
    global sidebar_stories
    for data in newsstories:
        htmlatag = data.find("h2", class_="title").find("a")
        headline = htmlatag.getText()
        url = htmlatag.get("href")
        d = {"headline" : headline,
             "url" : url}
        sidebar_stories.append(d)

def opinion_scrapeWebsites():
    global sidebar_stories
    foxnews = "http://www.foxnews.com/"
    page = urllib.urlopen(foxnews)
    r  = requests.get(foxnews)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    foundstories = soup.find(class_="collection collection-opinion has-load-more js-opinion").find_all("article")
    for article in foundstories:
        opinion_getTheGoodStuff(foundstories)
        
opinion_scrapeWebsites()
sidebar_articles = pd.DataFrame(sidebar_stories)
sidebar_articles.drop_duplicates(inplace=True)
sort_ranks = range(sidebar_articles.shape[0]+1)
del sort_ranks[0]
sidebar_articles["sort_rank"] = sort_ranks
sidebar_articles['date'] = datetime.today().strftime('%Y-%m-%d')
sidebar_articles['year'] = datetime.today().year
sidebar_articles['month'] = datetime.today().month
sidebar_articles['day'] = datetime.today().day
sidebar_articles['hour'] = datetime.today().hour
sidebar_articles['minute'] = datetime.today().minute

sidebar_articles['article_type'] = "opinion"

all_articles = pd.concat([articles,sidebar_articles],sort=False)

#harvesting article text
from selenium import webdriver
from time import sleep

article_text = []
driver = webdriver.Chrome("C:\Users\Duncan\Data Science\chromedriver.exe")

for url in all_articles.url:
    driver.get(url)
    sleep(1) #wait 1 second
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
    article_text.append(soup.find_all('p'))

driver.close()    
all_articles['article_text'] = article_text

foxnews_articles = pd.read_csv('foxnews.csv')
print "foxnews articles current shape:"
print foxnews_articles.shape[0]
print ""
conc = pd.concat([foxnews_articles,all_articles],sort=False)
conc.reset_index(inplace=True)
conc.drop(['index','Unnamed: 0'],axis=1,inplace=True)

conc.to_csv('foxnews.csv', encoding = 'utf-8')
print "foxnews new shape:"
print conc.shape[0]

foxnews articles current shape:
233

foxnews new shape:
286


In [8]:
all_articles

,headline,sort_rank,url,date,year,month,day,hour,minute,article_type,article_text
0,2020 Dem hopeful Biden makes a play for Texas,1,https://video.foxnews.com/v/6043573331001/,2019-06-01,2019,6,1,13,44,video,"[<p>Sign in to comment!</p>, <p>Jun. 01, 2019 ..."
1,Police identify all 12 victims of Virginia Bea...,1,https://www.foxnews.com/us/virginia-beach-poli...,2019-06-01,2019,6,1,13,44,us,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
2,"From politicos and celebs, Va. shooting brings...",2,https://www.foxnews.com/us/from-politicos-and-...,2019-06-01,2019,6,1,13,44,us,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
3,Ilhan Omar slammed for blaming NRA for Virgini...,2,https://www.foxnews.com/politics/ilhan-omar-sl...,2019-06-01,2019,6,1,13,44,politics,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
4,4:17,2,https://video.foxnews.com/v/6043540712001/,2019-06-01,2019,6,1,13,44,video,"[<p>Sign in to comment!</p>, <p>Jun. 01, 2019 ..."
5,How prevalent is mental illness in mass shooti...,2,https://www.foxnews.com/us/how-prevalent-is-me...,2019-06-01,2019,6,1,13,44,us,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
6,"Suspect who killed 12 taken down by 4 cops, ch...",2,https://www.foxnews.com/us/at-least-12-deaths-...,2019-06-01,2019,6,1,13,44,us,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
7,New Zealand’s new gun laws taking effect after...,2,https://www.foxnews.com/world/new-zealand-gun-...,2019-06-01,2019,6,1,13,44,world,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
9,Navy confirms request to ‘minimize’ USS John M...,1,https://www.foxnews.com/politics/navy-confirms...,2019-06-01,2019,6,1,13,44,politics,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
10,Explore some of our national treasures with Ab...,1,https://nation.foxnews.com/personalities/abby-...,2019-06-01,2019,6,1,13,44,nation,"[<p class=""Footer_legalText_2Ygr7"">Trademark &..."


In [10]:
conc.article_text[1]

'[<p class="copyright" data-optly-6dcb091b-a70c-4c25-b431-3878b10cee2e="">This material may not be published, broadcast, rewritten, or redistributed. \xc2\xa92019 FOX News Network, LLC. All rights reserved. All market data delayed 20 minutes.</p>, <p data-v-275a95fe="">Trump attorney Rudy Giuliani reacts to Mueller\'s remarks on Russia investigation.</p>, <p class="speakable">President Trump\'s personal attorney <a href="https://www.foxnews.com/transcript/giuliani-mueller-concluded-don-jr-did-nothing-wrong" target="_blank">Rudy Giuliani</a><a href="https://www.foxnews.com/person/g/rudy-giuliani" target="_blank"> </a>didn\'t hold back when it came to addressing <a href="https://www.foxnews.com/person/g/rudy-giuliani" target="_blank">Special Counsel Robert Mueller</a>\'s latest comments on the Russia investigation.</p>, <p class="speakable">Giuliani who appeared on "<a href="https://www.foxnews.com/category/topic/fox-news-flash" target="_blank">The Story with Martha MacCallum</a>" Wednes